<a href="https://colab.research.google.com/github/jennimotaa/Dados_com_Colab/blob/main/Codigos_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Códigos Google Colab.


# Carregar dados.

In [20]:
# Importar o comando pandas para dados.
import pandas as pd

In [21]:
df_carbon = pd.read_csv('/content/carbon_offsets.csv')
df_customers = pd.read_csv('/content/customers_data.csv')
df_rentals = pd.read_json('/content/rentals_data.json')
df_vehicles = pd.read_json('/content/vehicles_data.json')

In [22]:
# Pré-diagnóstico:

# O comando info() mostra o nome das colunas, a quantidade de dados por coluna e o tipo de dado.
# O comando head() exibe os primeiros 5 linhas de dados.
df_carbon.info()
df_carbon.head()

# O comando info() mostra o nome das colunas, a quantidade de dados por coluna e o tipo de dado.
# O comando head() exibe os primeiros 5 linhas de dados.
df_customers.info()
df_customers.head()

# O comando info() mostra o nome das colunas, a quantidade de dados por coluna e o tipo de dado.
# O comando head() exibe os primeiros 5 linhas de dados.
df_rentals.info()
df_rentals.head()

# O comando info() mostra o nome das colunas, a quantidade de dados por coluna e o tipo de dado.
# O comando head() exibe os primeiros 5 linhas de dados.
df_vehicles.info()
df_vehicles.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   year                2 non-null      int64
 1   carbon_offset_tons  2 non-null      int64
 2   cost_usd            2 non-null      int64
dtypes: int64(3)
memory usage: 180.0 bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   customer_id    4 non-null      object 
 1   customer_type  4 non-null      object 
 2   age            3 non-null      float64
 3   city           4 non-null      object 
 4   loyalty_score  4 non-null      int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 292.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  -

,vehicle_id,model,fuel_type,co2_g_per_km,category
0,V100,HB20,Gasolina,120.0,Econômico
1,V101,Corolla,Gasolina,160.0,Sedan
2,V102,Compass,Diesel,180.0,SUV
3,V103,Tesla Model 3,Elétrico,0.0,Elétrico
4,V104,Onix,Gasolina,NaN,Econômico


- Customers - Age com valor ausente.

- Rentals - Dailyrate com valor < 0 esta como string, data invalida, e formatação diferente.

- Vehicles - co2_g_per_km dado ausente.


# Identificação de dados

- Datas.
- Valores Negativos.
- DataTypes (Tipo de Dados).

# Correções:

In [23]:
# Customers - Age com valor ausente

media_idade = df_customers['age'].mean() # estou calculando a media
df_customers['age'] = df_customers['age'].fillna(media_idade)

In [24]:
# Rentals - daily_rate com valor < 0.

df_rentals.loc[df_rentals['daily_rate']<=0, 'daily_rate'] = pd.NA

# Rentals - usando o valor médio.

df_rentals['daily_rate'] = pd.to_numeric(df_rentals['daily_rate'], errors = "coerce")
media_daily_rate = df_rentals['daily_rate'].mean()
df_rentals['daily_rate'] = df_rentals['daily_rate'].fillna(media_daily_rate)

# Rentals - data invalida e formatacao diferente.

df_rentals['start_date'] = pd.to_datetime(df_rentals['start_date'], errors = "coerce")
df_rentals['end_date'] = pd.to_datetime(df_rentals['end_date'], errors = "coerce")
display(df_rentals)

,rental_id,customer_id,vehicle_id,start_date,end_date,city,daily_rate,distance_km
0,2001,C001,V100,2024-01-05,2024-01-08,São Paulo,180.0,320
1,2002,C002,V101,NaT,NaT,Curitiba,150.0,210
2,2003,C003,V102,NaT,2024-01-15,Salvador,220.0,0
3,2004,C004,V103,2024-02-01,2024-02-10,São Paulo,300.0,980
4,2005,C001,V104,2024-02-15,2024-02-16,Rio de Janeiro,212.5,120


In [25]:
# Vehicles - co2_g_per_km dado ausente (colocar media ou o mesmo valor da eco).
media_vehicles = df_vehicles['co2_g_per_km'].mean()
df_vehicles['co2_g_per_km'] = df_vehicles['co2_g_per_km'].fillna(media_vehicles)
display(df_vehicles)

,vehicle_id,model,fuel_type,co2_g_per_km,category
0,V100,HB20,Gasolina,120.0,Econômico
1,V101,Corolla,Gasolina,160.0,Sedan
2,V102,Compass,Diesel,180.0,SUV
3,V103,Tesla Model 3,Elétrico,0.0,Elétrico
4,V104,Onix,Gasolina,115.0,Econômico


# Engenharia de dados

- Calcule a duração de cada locação(em dias).
- Calcue a receita total por locação.
- Faça o merge com a base de veículos.
- Calcule a emissão de CO2 por locação.
- Extraia ano e mês de data de ínicio da locação.

In [26]:
# criar backup de dados

df_carbon_clean = df_carbon
df_rentals_clean = df_rentals
df_customer_clean = df_customers
df_vehicles_clean = df_vehicles

In [27]:
# Calcule a duração de cada locação(em dias).
df_rentals_clean['dias_locacao'] = (
    df_rentals_clean['end_date'] - df_rentals_clean['start_date']
).dt.days
display(df_rentals_clean)

,rental_id,customer_id,vehicle_id,start_date,end_date,city,daily_rate,distance_km,dias_locacao
0,2001,C001,V100,2024-01-05,2024-01-08,São Paulo,180.0,320,3.0
1,2002,C002,V101,NaT,NaT,Curitiba,150.0,210,NaN
2,2003,C003,V102,NaT,2024-01-15,Salvador,220.0,0,NaN
3,2004,C004,V103,2024-02-01,2024-02-10,São Paulo,300.0,980,9.0
4,2005,C001,V104,2024-02-15,2024-02-16,Rio de Janeiro,212.5,120,1.0


In [28]:
# Calcule a receita total por locacao, valor da diaria = daily_rate dias da loca * valor da diaria
df_rentals_clean['receita_total'] = df_rentals_clean['dias_locacao'] * df_rentals_clean['daily_rate']

In [29]:
# Faca o merge com a base de veiculos
df_consolidado = df_rentals_clean.merge(df_vehicles_clean, on = 'vehicle_id')
df_consolidado = df_consolidado.merge(df_customer_clean, on = 'customer_id')
display(df_consolidado)

,rental_id,customer_id,vehicle_id,start_date,end_date,city_x,daily_rate,distance_km,dias_locacao,receita_total,model,fuel_type,co2_g_per_km,category,customer_type,age,city_y,loyalty_score
0,2001,C001,V100,2024-01-05,2024-01-08,São Paulo,180.0,320,3.0,540.0,HB20,Gasolina,120.0,Econômico,Recorrente,35.0,São Paulo,90
1,2002,C002,V101,NaT,NaT,Curitiba,150.0,210,NaN,NaN,Corolla,Gasolina,160.0,Sedan,Novo,28.0,Curitiba,60
2,2003,C003,V102,NaT,2024-01-15,Salvador,220.0,0,NaN,NaN,Compass,Diesel,180.0,SUV,Novo,36.0,Salvador,50
3,2004,C004,V103,2024-02-01,2024-02-10,São Paulo,300.0,980,9.0,2700.0,Tesla Model 3,Elétrico,0.0,Elétrico,Corporativo,45.0,São Paulo,95
4,2005,C001,V104,2024-02-15,2024-02-16,Rio de Janeiro,212.5,120,1.0,212.5,Onix,Gasolina,115.0,Econômico,Recorrente,35.0,São Paulo,90


In [30]:
# Calcule a emissão de CO2 por locação.
df_consolidado['co2_emitido'] = df_consolidado['co2_g_per_km'] * df_consolidado['distance_km']

In [31]:
# Extrair ano e mês da data de início da locação para extrair as informaçõess da data que temos:

# .dt.day - dia
# .dt.weekday - dia da semana
# .dt.month - mes
# .dt.year - ano

df_consolidado['dia'] = df_consolidado['start_date'].dt.day
df_consolidado['dia da semana'] = df_consolidado['start_date'].dt.weekday
df_consolidado['mes'] = df_consolidado['start_date'].dt.month
df_consolidado['ano'] = df_consolidado['start_date'].dt.year
display(df_consolidado)

,rental_id,customer_id,vehicle_id,start_date,end_date,city_x,daily_rate,distance_km,dias_locacao,receita_total,...,category,customer_type,age,city_y,loyalty_score,co2_emitido,dia,dia da semana,mes,ano
0,2001,C001,V100,2024-01-05,2024-01-08,São Paulo,180.0,320,3.0,540.0,...,Econômico,Recorrente,35.0,São Paulo,90,38400.0,5.0,4.0,1.0,2024.0
1,2002,C002,V101,NaT,NaT,Curitiba,150.0,210,NaN,NaN,...,Sedan,Novo,28.0,Curitiba,60,33600.0,NaN,NaN,NaN,NaN
2,2003,C003,V102,NaT,2024-01-15,Salvador,220.0,0,NaN,NaN,...,SUV,Novo,36.0,Salvador,50,0.0,NaN,NaN,NaN,NaN
3,2004,C004,V103,2024-02-01,2024-02-10,São Paulo,300.0,980,9.0,2700.0,...,Elétrico,Corporativo,45.0,São Paulo,95,0.0,1.0,3.0,2.0,2024.0
4,2005,C001,V104,2024-02-15,2024-02-16,Rio de Janeiro,212.5,120,1.0,212.5,...,Econômico,Recorrente,35.0,São Paulo,90,13800.0,15.0,3.0,2.0,2024.0
